## Importação das Bibiliotecas

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

In [3]:
# Crie um SparkSession
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura da View Participantes

## Parametrizando a conexão do Banco SQL SERVER

In [5]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "Participantes"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [6]:
#Conectando via jdbc
#No método option, foi necessário passar os argumentos ssl e trustservercertificate como true, por se tratar de um banco local sem SSL.
try:
  query = "select * from dbo.VW_PARTICIPANTES_ENEM"

  dfParticipantes = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", f"({query}) as alias_name") \
        .option("user", username) \
        .option("password", password).load()
      
  dfParticipantes.createOrReplaceTempView("ParticipantesEnemAGR")  
 # print(dfParticipantes.count())
except ValueError as error :
    print("Connector write failed", error)


In [7]:
dfAGR = spark.sql("""
    SELECT
        NU_ANO,
        FAIXA_ETARIA,
        SEXO,
        ESTADO_CIVIL,
        raca,
        NACIONALIDADE,
        SITUACAO_CONCLUSAO,
        ANO_CONCLUSAO,
        ESCOLA,
        ENSINO,
        TREINEIRO,
        COUNT(1) as QTD
    FROM ParticipantesEnemAGR
    GROUP BY
        NU_ANO,
        FAIXA_ETARIA,
        SEXO,
        ESTADO_CIVIL,
        raca,
        NACIONALIDADE,
        SITUACAO_CONCLUSAO,
        ANO_CONCLUSAO,
        ESCOLA,
        ENSINO,
        TREINEIRO
""")


In [8]:
dfPreencher = dfAGR.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER

In [9]:
# Nome da nova tabela que será criada
table_name = 'ParticipantesAGR'

In [10]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [12]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [13]:
print(f'Tabela {table_name} criada com sucesso no banco de dados.')


Tabela ParticipantesAGR criada com sucesso no banco de dados.
